In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Normalization
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten
from keras import layers
pd.set_option('display.max_rows', 50)
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping 

import random


2021-09-02 09:44:10.850714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-02 09:44:10.850766: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df_merged = pd.read_csv("rnn_model_df_31.csv")

In [3]:
df_merged = df_merged[df_merged["reply_count"].notna()] 

In [4]:
df_merged = df_merged.replace(np.nan, 0)

In [5]:
df_merged = df_merged.drop(columns = ["reply_count", "avg_followers_count", "avg_following_count" ] )

In [6]:
df_merged["tweet_date"] = pd.to_datetime(df_merged["tweet_date"])

df_merged = df_merged.groupby([pd.Grouper(key='tweet_date',freq='D'), "party"]).agg({
#     "reply_count": "sum", 
    "retweet_count": "sum",
    "like_count": "sum",
#     "avg_len_of_tweet": "mean" ,
#     "avg_followers_count": "mean" ,
#     "avg_following_count": "mean" ,
    "avg_user_tweet_count" :"mean",
    "avg_ff_ratio" :"mean",
    "share_of_tweets" :"mean",
    "share_unique_users": "mean",
    "weighted_sentiment" : "mean",
    "share_of_positive_tweets": "mean",
    "share_of_negative_tweets" : "mean",
    "poll": "mean"}) 

In [7]:
df_real_true = df_merged.iloc[-7:]

In [8]:
df_merged = df_merged.iloc[:-7]

In [9]:
df_merged_final_test = df_merged.iloc[-7:]

In [10]:
df_merged_final = df_merged.iloc[:-7]

In [11]:
num_features = [ "retweet_count", "like_count", "avg_user_tweet_count","avg_ff_ratio"]
remaing_columns = ["share_of_tweets","share_unique_users","weighted_sentiment","share_of_positive_tweets","share_of_negative_tweets"]

In [12]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()

mm_scaler.fit(df_merged_final[num_features])

numerical_train_scaled = mm_scaler.transform(df_merged_final[num_features])
numerical_test_scaled = mm_scaler.transform(df_merged_final_test[num_features])

In [13]:
df_merged_poll = df_merged_final["poll"] / 100
df_merged_poll_test = df_merged_final_test["poll"] / 100

In [14]:
df_numerical_train_scaled = pd.DataFrame(numerical_train_scaled, columns = num_features, index = df_merged_final.index)


In [15]:
df_train_scaled = pd.concat([df_numerical_train_scaled,df_merged_final[remaing_columns], pd.DataFrame(df_merged_poll, columns = ["poll"], index = df_merged_final.index)] , axis = 1)

In [16]:
df_test_scaled = pd.concat([pd.DataFrame(numerical_test_scaled, columns = num_features, index = df_merged_final_test.index),df_merged_final_test[remaing_columns], pd.DataFrame(df_merged_poll_test, columns = ["poll"], index = df_merged_final_test.index)] , axis = 1)

In [17]:
def subsequence(df, length, start = 0):
    last_possible_start = len(df) - length 
    X = df[start:start + length]
    y = df["poll"][(start + length ) : (start + length + 7)]
    
    return X, y

In [18]:
def multiple_subsequences(df, length):
    
    list_of_X = []
    list_of_y = []
    
    number_of_subsequences = round(len(df) /  7 - (length/7))
    start = 0
    for num in range(number_of_subsequences):
        temporary_X, temporary_y = subsequence(df, length, start)
        list_of_X.append(temporary_X)
        list_of_y.append(temporary_y)
        start = start + 7
    
    return list_of_X, list_of_y

    
    
    

In [19]:
X_train , y_train = multiple_subsequences(pd.DataFrame(df_train_scaled), 21 )


In [20]:
y_train

[tweet_date  party 
 2021-07-26  AFD       0.11
             CDU       0.27
             FDP       0.13
             GRUENE    0.18
             LINKE     0.07
             OTHER     0.07
             SPD       0.17
 Name: poll, dtype: float64,
 tweet_date  party 
 2021-07-27  AFD       0.120
             CDU       0.270
             FDP       0.130
             GRUENE    0.175
             LINKE     0.060
             OTHER     0.070
             SPD       0.175
 Name: poll, dtype: float64,
 tweet_date  party 
 2021-07-28  AFD       0.1012
             CDU       0.2800
             FDP       0.1225
             GRUENE    0.1988
             LINKE     0.0700
             OTHER     0.0700
             SPD       0.1575
 Name: poll, dtype: float64,
 tweet_date  party 
 2021-07-29  AFD       0.1012
             CDU       0.2800
             FDP       0.1225
             GRUENE    0.1988
             LINKE     0.0700
             OTHER     0.0700
             SPD       0.1575
 Name: poll, d

In [21]:
X_train = np.array(X_train)
# X_test = np.array(X_test)
y_train = np.array(y_train)
# y_test = np.array(y_test)


In [22]:
model = Sequential()

model = Sequential()
model.add(layers.LSTM(units=128, activation='tanh', return_sequences=True))
model.add(layers.LSTM(units=64, activation='tanh', return_sequences=False))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(7, activation='linear'))


2021-09-02 09:44:13.193332: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-02 09:44:13.193372: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-02 09:44:13.193389: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-69AIGBNI): /proc/driver/nvidia/version does not exist
2021-09-02 09:44:13.193645: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
%%time

es = EarlyStopping(patience = 10 , restore_best_weights=True)


model.compile(loss='mse', optimizer='adam', metrics=['mae'])

history = model.fit(X_train, y_train ,
          epochs=0, 
          batch_size=1, 
          verbose=1, 
          validation_split=0.2,
          shuffle = False,
          callbacks=[es]
     )



CPU times: user 53.3 ms, sys: 1.66 ms, total: 55 ms
Wall time: 48.8 ms


In [ ]:
df_merged_final_day = np.array(df_test_scaled)
df_merged_final_day = np.expand_dims(df_merged_final_day, axis = 0)

In [ ]:
df_prediction = pd.DataFrame(model.predict(df_merged_final_day)).rename( \
                        columns = {0:'AFD',1:'CDU',2:'FDP',3: 'GRUENE',4: 'LINKE', 5:'OTHER', 6: 'SPD'}).round(3)
df_prediction.index = ["pred"]

df_prediction

In [ ]:
df_prediction_2 = pd.DataFrame((df_prediction.iloc[0]) / np.sum(df_prediction.iloc[0])).T
df_prediction_2 = df_prediction_2.round(4)

In [ ]:
df_prediction_2

In [ ]:
df_true = pd.DataFrame(df_real_true["poll"]).T/100

In [ ]:
df_true = pd.DataFrame(df_true.iloc[0]).reset_index().groupby(by = "party").mean().T

In [ ]:
df_true

In [ ]:
df_plotting_spd = df_train_scaled.reset_index().loc[df_train_scaled.reset_index()["party"] == "SPD"].drop(columns = "party")
df_plotting_cdu = df_train_scaled.reset_index().loc[df_train_scaled.reset_index()["party"] == "CDU"].drop(columns = "party")

In [ ]:
new_df = df_plotting_spd.melt(id_vars='tweet_date', 
                 value_name='val', 
                 var_name='type')

sns.lineplot(x='tweet_date', y='val', hue='type', data=new_df)

In [ ]:
new_df = df_plotting_cdu.melt(id_vars='tweet_date', 
                 value_name='val', 
                 var_name='type')

sns.lineplot(x='tweet_date', y='val', hue='type', data=new_df)